In [20]:
import pickle
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn import metrics
plt.style.use('seaborn')
sns.set(style="white")

In [21]:
HOLDOUT_FILENAME = "kc_house_data_test_features.csv"
PICKLENAME="model+cols.p"
CREATECSVNAME = "full_predictions.csv"
picdump = pickle.load( open( "model+cols2.p", "rb" ) )
picdump

[LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False),
 Index(['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
        'waterfront', 'view', 'grade', 'sqft_above', 'sqft_basement',
        ...
        'sqft_lot15 x98039', 'sqft_lot15 x98040', 'sqft_lot15 x98102',
        'sqft_lot15 x98105', 'sqft_lot15 x98109', 'sqft_lot15 x98112',
        'sqft_lot15 x98119', 'sqft_lot15 x98168', 'sqft_lot15 x98177',
        'sqft_lot15 x98199'],
       dtype='object', length=477)]

In [22]:
from sklearn.model_selection import train_test_split
def df_cleaner(csv_file_name, random_state=None, split=False):
    df = pd.read_csv(csv_file_name)
    
    # turn zipcode into into onehot encoding - easier than binning by lat/long
    df = pd.concat([df,pd.get_dummies(df["zipcode"],drop_first=True)],axis=1)
    df = df.drop(["zipcode"],axis=1)
    
    # drop the original index column and id because using id as index is
    # annoying for scaling
    #df.set_index("id",inplace=True)
    df.drop("Unnamed: 0",axis=1,inplace=True)
    df.drop("id",axis=1,inplace=True)
    
    # can't use datetime form so we convert to timestamp
    df["date"] = pd.to_datetime(df.date).apply(lambda x: x.timestamp())
    
    # removing price so we can plug it into the splitter
    if split:
        target = df["price"]
        features = df.drop("price", axis=1)

    if split:
        x_train, x_test, y_train, y_test = train_test_split(features, target, random_state=34,test_size=0.2)
        return x_train, x__test, y_train, y_test
    return df


x_holdout = df_cleaner(HOLDOUT_FILENAME)

x_holdout.columns = map(lambda x: "x" + str(x) if type(x)==int else x, x_holdout.columns)


In [23]:
n=2
poly = PolynomialFeatures(degree=n, include_bias=False, interaction_only=True)
poly_data = poly.fit_transform(x_holdout)
poly_columns = poly.get_feature_names(x_holdout.columns)
df_poly = pd.DataFrame(poly_data, columns=poly_columns)
selected_cols = df_poly[picdump[1]]
selected_cols.shape

(4323, 477)

In [24]:
predictions = picdump[0].predict(selected_cols)

In [27]:
preddf = pd.DataFrame(predictions)
preddf.to_csv(CREATECSVNAME,header=False)